# Stack Overflow Webscraping Project

Description here

In [49]:
# Import libraries and modules
import pandas as pd
import time
import requests
from bs4 import BeautifulSoup

## Webscraping Stack Overflow

In [58]:
# Created an empty dataframe with the desired columns
df = pd.DataFrame(
    columns = [["company", "industry", "size", "year_founded", "status", "follower_count", "tech_stack", "benefits"]]
)

display(df)

,company,industry,size,year_founded,status,follower_count,tech_stack,benefits


In [53]:
# Created a function that collects spcific data points from stack overflow
def add_data_to_df(soupy):
    company_name = soupy.find("h1").text.strip() # Extract --> Company Name

    
    about_company = soupy.find_all("span", class_ = "d-block")


    indices = [12,13,14,15,16] # The indicies 12,13,14,15,and 16 represents industy, size, year founded, status, and followers
    values = []

    for index in indices: # This loop ignores IndexErrors when running the function for certain companies that are missing data
        try:
            values.append(about_company[index].text.strip())
        except IndexError:
            values.append(None) 

    industry = values[0]
    size = values[1]
    year_founded = values[2]
    status = values[3]
    followers = values[4]

    tech_skills = soupy.find_all("a", class_ = "flex--item s-tag no-tag-menu") # Searched for the company's tech stack
    tech_stack = [] # Created a list containing all relevant tech skills 

    for item in range(len(tech_skills)):
        tech_stack.append(tech_skills[item].text.strip())


    benefits_list = soupy.find_all("div", class_="flex--item pl8 pt2 fw-normal fs-body2 fc-black-700") # Searched for company benefits
    benefits = [] # Created a list containing all listed company benefits

    for item in range(len(benefits_list)):
        benefits.append(benefits_list[item].text.strip())
    
    
    # Appended all webscrapped elements into a list "new_row"

    new_row = [] 

    new_row.append(company_name)
    new_row.append(industry)
    new_row.append(size)
    new_row.append(year_founded)
    new_row.append(status)
    new_row.append(followers)
    new_row.append(tech_stack)
    new_row.append(benefits)

    df.loc[len(df.index)] = new_row # Added the list as a new row in the dataframe

In [42]:
# Accessing each link 

def scrape_page(page_link):

    company_list = page_link.find_all("a", class_="s-link", href=lambda href: href and "/jobs/companies" in href) # Accessed each <a> tag that includes href and "/jobs/companies"

    company_links = []

    for names in range(len(company_list)):
        company_links.append("https://stackoverflow.com" + company_list[names]["href"]) # The extracted links combines with "https://stackoverflow.com" generates the full link for each company's page on stack overflow


    # Created a for loop that loops through each company page at least three times
    for link in range(len(company_links)):  
        response = requests.get(company_links[link])
        max_attempts = 3 

        while max_attempts > 0:

            if response.status_code == 200:
                print("Successful connection")
                soupy = BeautifulSoup(response.text, "html")
                add_data_to_df(soupy)
                break

            elif response.status_code == 429:
                print("Response 429, reattempting...")
                time.sleep(5) # If the server is handling too many request --> wait five seconds and reattempt connection
                max_attempts -=1

            elif response.status_code == 403:
                print("Access Denied")
                break
            
            else:
                print(response.status_code)
                break

In [59]:
# Created a for loop that checks the connection of each page and scrapes data from each company's profile page through accessing their individual links

connection_attempts = 3
max_pages = 7

for page in range(1, max_pages + 1):
    url = f"https://stackoverflow.com/jobs/companies?pg={page}"
     
    while connection_attempts > 0:
        response = requests.get(url)

        if response.status_code == 200: # used a similar for loop structure to the "scrape_page" function created above
            print(f"Successfully connected to page:{page}")
            soup = BeautifulSoup(response.text, "html")
            scrape_page(soup)
            break

        elif response.status_code == 429:
            print("Response [429]: Reattempting to connect")
            time.sleep(5)
            connection_attempts -= 1

        elif response.status_code == 403:
                print("Response [403]: Access denied") 
                break
        else:
            print("Invalid link")
            break
    if connection_attempts == 0:
        print("Max number of attempts excceded")

Successfully connected to page:1
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successfully connected to page:2
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successfully connected to page:3
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successfully connected to page:4
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful connection
Successful

In [60]:
# Show the extracted data
display(df)

,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
0,Paylocity,"Human Resources, Payroll, Software Development...",5k-10k employees,1997,Public,143,"[aws, api-gateway, .net, c#, sql-server, react...",[100% Remote Opportunities in Product & Tech T...
1,GFT,"Banking, Cloud-Based Solutions, Financial Tech...",1k-5k employees,1987,Private,160,"[java, angular, .net, react, sql, python, rest...",[Worksmile - a cafeteria system. Monthly accou...
2,Zoi TechCon GmbH,"Cloud Computing, Cloud Services, IT Consulting",201-500 employees,2017,Private,91,"[aws, azure, google-cloud-platform, react, .ne...",[Germany job ticket (payment of the total amou...
3,Intuit,"Computer Software, Financial Technology",10k+ employees,1983,Public,642,"[java, kotlin, scala, play, spark, react, aws,...","[Well-being for Life Reimbursement Program, Em..."
4,Discover Financial Svcs,"Banking, Financial Services",10k+ employees,1986,Public,151,"[javascript, tableau-api, powerbi, qlikview, t...","[Annual Leave - 5 to 7 weeks, Tuition Reimburs..."
...,...,...,...,...,...,...,...,...
59,UBS,"Financial Services, Financial Technology",10k+ employees,Private,102,None,"[javascript, java, kotlin, kubernetes, cicd, p...",[Working with cutting edge tech and agile prac...
60,ZEISS Group,"Machine Learning, Medical Devices, Virtual Rea...",10k+ employees,1846,Private,218,"[angular, swift, c#, azure, javascript, python...","[Flexible working time models, Open, dialogue-..."
61,Citi,"Finance, Financial Technology, Risk Management",10k+ employees,Public,192,None,"[java, python, api, management, c#, ai, cloud-...","[Medical, Dental and Vision coverage, Paid Tim..."
62,KfW,5k-10k employees,1948,Public,52,None,"[java, pl1, javascript, python, sap, c#, c++, ...","[Marktgerechte Vergütung, Betriebliche Altersv..."


## Data Cleaning and Feature Engineering

In [61]:
# Created a csv file based on the extracted data
df.to_csv("companies.csv", index = False)
df = pd.read_csv("companies.csv")
display(df)

,company,industry,size,year_founded,status,follower_count,tech_stack,benefits
0,Paylocity,"Human Resources, Payroll, Software Development...",5k-10k employees,1997,Public,143,"['aws', 'api-gateway', '.net', 'c#', 'sql-serv...",['100% Remote Opportunities in Product & Tech ...
1,GFT,"Banking, Cloud-Based Solutions, Financial Tech...",1k-5k employees,1987,Private,160,"['java', 'angular', '.net', 'react', 'sql', 'p...",['Worksmile - a cafeteria system. Monthly acco...
2,Zoi TechCon GmbH,"Cloud Computing, Cloud Services, IT Consulting",201-500 employees,2017,Private,91,"['aws', 'azure', 'google-cloud-platform', 'rea...",['Germany job ticket (payment of the total amo...
3,Intuit,"Computer Software, Financial Technology",10k+ employees,1983,Public,642,"['java', 'kotlin', 'scala', 'play', 'spark', '...","['Well-being for Life Reimbursement Program', ..."
4,Discover Financial Svcs,"Banking, Financial Services",10k+ employees,1986,Public,151,"['javascript', 'tableau-api', 'powerbi', 'qlik...","['Annual Leave - 5 to 7 weeks', 'Tuition Reimb..."
...,...,...,...,...,...,...,...,...
59,UBS,"Financial Services, Financial Technology",10k+ employees,Private,102,NaN,"['javascript', 'java', 'kotlin', 'kubernetes',...",['Working with cutting edge tech and agile pra...
60,ZEISS Group,"Machine Learning, Medical Devices, Virtual Rea...",10k+ employees,1846,Private,218,"['angular', 'swift', 'c#', 'azure', 'javascrip...","['Flexible working time models', 'Open, dialog..."
61,Citi,"Finance, Financial Technology, Risk Management",10k+ employees,Public,192,NaN,"['java', 'python', 'api', 'management', 'c#', ...","['Medical, Dental and Vision coverage', 'Paid ..."
62,KfW,5k-10k employees,1948,Public,52,NaN,"['java', 'pl1', 'javascript', 'python', 'sap',...","['Marktgerechte Vergütung', 'Betriebliche Alte..."


In [62]:
df.isnull().sum()

company            0
industry           0
size               0
year_founded       0
status             0
follower_count    13
tech_stack         0
benefits           0
dtype: int64

In [ ]:
# Separated the "tech_stack" column into multiple columns organized by type of tech
# New Columns: "Cloud Services and Infrastructure", "Programing Languages and Frameworks", "Databases and Data Technologies"
